In [16]:
from PIL import Image
import pytesseract
import cv2
import os
from imutils import contours

In [17]:
def get_selected_color_imgs(filename,color="red"):
    color_coding = {"red":0,"green":1,"blue":2}
    width, height = 150,150
    image = Image.open(filename)
    if(image.mode != "RGB"):
        image = image.convert('RGB')
    image = image.resize((width, height),resample = 0)
    pixels = image.getcolors(width * height)
    sorted_pixels = sorted(pixels, key=lambda t: t[0])
    dominant_color = sorted_pixels[-1][1]
    max_color = max(dominant_color)
    max_color_index = dominant_color.index(max_color)
    if (max_color_index == color_coding[color] and max_color > 0):
        return 1
    else:
        return 0

In [18]:
def get_digits_in_img2(filename):
    image_obj = cv2.imread(filename)
    gray = cv2.cvtColor(image_obj, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 5))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    text = get_digits_in_img(thresh)
    if(text == "" or not any(char.isdigit() for char in text)):
        text =  pytesseract.image_to_string(thresh, config='--psm 10')
    if(text == "" or not any(char.isdigit() for char in text)):
        text =  pytesseract.image_to_string(thresh, config='--psm 6')
    #print("With Contours , Text is: "+text)
    digits = []
    for char in text:
        if(char.isdigit()):
            digits.append(char)
    if(len(digits)>0):
        digits = "".join([str(digit) for digit in digits])
        return int(digits)
    else:
        return None

In [21]:
def get_digits_in_img(thresh):
    thresh_im_area = thresh.shape[0]*thresh.shape[1]
    im2, cnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    digitCnts = []
    for i,c in enumerate(cnts):
        (x, y, w, h) = cv2.boundingRect(c)
        cont_area = abs((w-x)*(h-y))
        if(cont_area < thresh_im_area-500 and hierarchy[0][i][-1] == 0):
            digitCnts.append(c)
    digits = ""
    if(len(digitCnts) > 0):
        digitCnts = contours.sort_contours(digitCnts,method="left-to-right")[0]
        for c in digitCnts:
            (x, y, w, h) = cv2.boundingRect(c)
            roi = thresh[y:y + h, x:x + w]
            scale_percent = 60 # percent of original size
            width = int(roi.shape[1] * scale_percent / 100)
            height = int(roi.shape[0] * scale_percent / 100)
            dim = (width, height)
            # resize image
            resized = cv2.resize(roi, dim, interpolation = cv2.INTER_AREA)
            #plt.imshow(resized2,cmap="gray")
            constant= cv2.copyMakeBorder(resized,45,45,45,45,cv2.BORDER_CONSTANT,value=(255,255,255))
            text = pytesseract.image_to_string(constant, config='--psm 6')
            digits = digits+text 
    return digits

In [22]:
images_dir = "Images"
sum_of_green_digits = 0

for img in os.listdir(images_dir):
    if not img.startswith('.'):
        is_green = get_selected_color_imgs(os.path.join(images_dir,img),"green")
        if(is_green == 1):
            curr_num = get_digits_in_img2(os.path.join(images_dir,img))
            print("{} output is {}.".format(img,curr_num))
            if(not curr_num is None):
                sum_of_green_digits += curr_num

print("Sum of the numbers is:",sum_of_green_digits)

TesseractNotFoundError: tesseract is not installed or it's not in your path